In [ ]:
## INITIALIZATION

In [1]:
import pandas as pd
import numpy as np
import json

import time

from itertools import chain
import os
import sys
from glob import glob
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode
from pyspark.sql import functions as sf
from pyspark.sql.types import *

from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
from functools import reduce



# suppress warnings for readability
import warnings
warnings.filterwarnings('ignore')
# turn off scientific notation in pandas objects
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [2]:
# Initialize spark session
spark = SparkSession.builder.appName('SafeGraph').config("spark.driver.memory", "15g").getOrCreate()

In [3]:
# read in 2019 ACS 5-year population data
census_acs = "/Users/esrieves/Documents/school/Research/foot_traffic/data/census/safegraph_open_census_data_2019/data/cbg_b01.csv"

In [ ]:
## FUNCTIONS

In [4]:
## parser function
# input: JSON dictionary element
# output: parsed JSON element ready for explosion
def parser(element):
    parsed = json.loads(element)

    if parsed is not None:
        return parsed
    else:
        return None

In [5]:
## pattern/panel files
# pattern: for each MSA, gets the file path and then 'walks' through all files in the subdirectory, creating a dataframe of all files
# panel: does the same for each MSA's panel data
# get patterns files for each MSA
def get_pattern_files(MSA):
    file_path = "/Users/esrieves/Documents/school/Research/foot_traffic/data/Inputs/%s_MSA" %MSA
    file_ext_patterns = "core_poi-patterns.csv.gz"
    
    all_pattern_files = [file
                for path, subdir, files in os.walk(file_path)
                for file in glob(os.path.join(path, file_ext_patterns))]
    
    return(all_pattern_files)

def get_panel_files(MSA):
    file_path = "/Users/esrieves/Documents/school/Research/foot_traffic/data/Inputs/%s_MSA" %MSA
    file_ext_panel = "home_panel_summary.csv"

    all_panel_files = [file
                for path, subdir, files in os.walk(file_path)
                for file in glob(os.path.join(path, file_ext_panel))]
    return(all_panel_files)

In [6]:
## clean pattern files function
# reads in all csv files as a spark DF (w header, inferred schema); drops null on important columns; replaces leading 0; edits date;
# trims NAICS to 2 #s, and selects important columns
def read_clean_patterns(all_csv_files):
    df = spark.read.option('header', 'True') \
        .option('inferSchema','True') \
        .option('escape', "\"") \
        .csv(all_csv_files)
    
    df = df.dropna(subset=["visitor_home_cbgs", "poi_cbg", "placekey"])
    df = df.where("visitor_home_cbgs!='{}'")

    # Add leading zero to poi_cbg column (lost in csv format)
    # Remove additional NAICS code digits > 2
    # Date range starts the last day of the month at 11 pm.. switching to the first day of the full month
    # e.g. 5/31 to 6/1
    df = df.withColumn("dest_cbg", sf.format_string("%012d","poi_cbg"))\
        .withColumn("naics", df.naics_code.substr(1,2))\
        .withColumn('date_start',sf.date_format(sf.date_add('date_range_start',1),"MM-yyyy"))\
        .withColumnRenamed('region','state')\
        .select('placekey', 'naics', 'state', 'date_start', 'raw_visitor_counts', 'raw_visit_counts', 'poi_cbg', 'visitor_home_cbgs',
               'bucketed_dwell_times')
    
    return(df)

In [7]:
def filter_workers(patterns_df):
    # Implement parser to parse and explode bucketed dwell time column
    jsonudf = udf(parser, MapType(StringType(), IntegerType()))
    
    # parse then explode dwell time column to get > 240 minutes of dwelling (assumption: these are workers)
    dwell_parsed = patterns_df.withColumn("parsed_dwell_times", jsonudf("bucketed_dwell_times"))
    worker_cbgs_exploded = dwell_parsed.select("placekey","raw_visit_counts", "date_start","state","visitor_home_cbgs", "poi_cbg", "naics",explode("parsed_dwell_times"))
    
    # filter 'workers' (> 240 min at poi)
    # determine percent of visits (not visitors, bc bucketed dwell times is based on visits not visitors) > 240 min to determine percent workers
    worker_cbgs_exploded = worker_cbgs_exploded.filter(worker_cbgs_exploded.key == ">240")\
        .withColumn("pct_worker",worker_cbgs_exploded.value/worker_cbgs_exploded.raw_visit_counts)
    
    # new df for analysis -- mostly visitors
    # created and commented out mostly workers in case it's of future interest
    #mostly_workers = worker_cbgs_exploded.where("pct_worker >= 0.3")
    mostly_visitors = worker_cbgs_exploded.where("pct_worker < 0.3")\
        .select('placekey', 'naics', 'state', 'date_start', 'poi_cbg', 'visitor_home_cbgs')
    
    return(mostly_visitors)

In [8]:
## census process function
# reads in census data, maps state fips to state name to get population
def census_process(census_dir):
    df = spark.read.option('header', 'True') \
        .option('inferSchema','True') \
        .option('escape', "\"") \
        .csv(census_dir)
    
    # select cbg and population columns, rename
    df = df.select('census_block_group', 'B01001e1')\
        .withColumnRenamed('B01001e1','population')
    
    fips_state_map = {
        '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
        '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
        '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
        '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
        '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
        '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
        '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
        '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
        '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
        '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
        '56': 'WY', '72': 'PR'
    }
    
    mapping_expr = sf.create_map([sf.lit(x) for x in chain(*fips_state_map.items())])
    
    # replace leading zero to ensure all states match
    df = df.withColumn("cbg", sf.format_string("%012d","census_block_group"))
    
    # create state fips column to match cbg with state name
    df = df.withColumn('state_fips',df.cbg.substr(1,2))\
        .withColumn("state", mapping_expr[sf.col("state_fips")])\
        .groupby('state').agg(sf.sum('population').alias('state_pop'))
    
    return(df)

In [9]:
## read home panel function
# read home_panel_summary file and summarize by date and state to calculate number of devices residing (used in aggregation)
def read_home_panel(all_csv_files_panel):
    df = spark.read.option('header', 'True') \
        .option('inferSchema','True') \
        .option('escape', "\"") \
        .csv(all_csv_files_panel)


    df = df.withColumn('month_2dig',sf.format_string("%02d","month"))\
        .withColumn('date_start',sf.concat_ws('-',sf.col('month_2dig'),sf.col('year')))\
        .select("*",sf.upper(sf.col('region')).alias('state'))\
        .groupby(['state', 'date_start'])\
        .agg(sf.sum('number_devices_residing').alias('number_of_devices_residing'))
    
    return(df)
 

In [10]:
## function to normalize and aggregate pattern data using census and panel data inputs
def normalize_and_aggregate(home_panel_df,census_df,patterns_df):
    df = home_panel_df.join(census_df, on = "state")
    # create state multiplier column
    df = df.withColumn("state_multiplier", df.state_pop/df.number_of_devices_residing)
    df_joined = df.join(patterns_df, on = ['date_start','state'])
    
    # Implement parser to parse and explode visitor home cbgs
    jsonudf = udf(parser, MapType(StringType(), IntegerType()))
    
    # parse then explode visitor home cbgs to get sender cbg
    visitor_home_cbgs_parsed = df_joined.withColumn("parsed_visitor_home_cbgs", jsonudf("visitor_home_cbgs"))
    visitor_home_cbgs_exploded = visitor_home_cbgs_parsed.select("placekey", "poi_cbg", "naics",
                                                             "date_start","state_multiplier",
                                                             explode("parsed_visitor_home_cbgs"))
    
    # use state multiplier to normalize visitor values and clean data (drop a few columns)
    agg_df = visitor_home_cbgs_exploded.withColumn("sender_cbg", visitor_home_cbgs_exploded.key) \
        .withColumn('norm_visitors',(sf.round((visitor_home_cbgs_exploded.value*visitor_home_cbgs_exploded.state_multiplier),0)))\
        .drop("placekey")\
        .drop("key")\
        .drop("value")\
        .drop("state_multiplier")\
        .drop("dest_tract")
    
    # Aggregate visitors based on sender and destination cbgs, NAICS code
    visitor_flows = agg_df.groupby(["sender_cbg","date_start","naics"])\
        .agg(sf.sum("norm_visitors").alias("monthly_visitors_per_naics_tracts_NORMALIZED"))
    
    return(visitor_flows)



In [ ]:
## RUN functions

In [25]:
census_data = census_process(census_acs)

# read and clean patterns and panels
pattern_files = get_pattern_files("Baltimore")
clean_patterns = read_clean_patterns(pattern_files)

# filter out predominately 'worker' pois (30%+ visits are > 6 hour duration)
visitor_patterns = filter_workers(clean_patterns)

panel_files = get_panel_files("Baltimore")
clean_panels = read_home_panel(panel_files)
    
print("finished file accumulation, starting flows")
    
output_data = normalize_and_aggregate(clean_panels,census_data,visitor_patterns)
    
print("finished with flows")
output_data.show()
print("finished writing file")

finished file accumulation, starting flows
finished with flows
+------------+----------+-----+--------------------------------------------+
|  sender_cbg|date_start|naics|monthly_visitors_per_naics_tracts_NORMALIZED|
+------------+----------+-----+--------------------------------------------+
|100030166042|   02-2018|   61|                                        86.0|
|240419605022|   06-2021|   72|                                      1587.0|
|240135120001|   06-2021|   72|                                      3991.0|
|245101402002|   06-2021|   61|                                       365.0|
|240276012015|   06-2021|   61|                                      1914.0|
|245102606056|   06-2021|   72|                                      3703.0|
|240135010012|   06-2021|   71|                                      3786.0|
|240054205001|   06-2021|   44|                                      9565.0|
|100030139042|   06-2021|   62|                                        73.0|
|245102709033

In [11]:
msas = ['NYC','LA','Chicago','Dallas','Houston','DC','Philadelphia','Miami','Atlanta','Boston','Phoenix','SanFrancisco',
       'Riverside','Detroit','Seattle','Minneapolis','SanDiego','Tampa','Denver','Baltimore']


# only need to run census data once
census_data = census_process(census_acs)

# for loop to run functions on all msas in list
for msa in msas:
    # start timing
    start_time = time.time()
    
    msa_str = str(msa)
    print(msa_str)
    
    # read and clean patterns data
    pattern_files = get_pattern_files(msa_str)
    clean_patterns = read_clean_patterns(pattern_files)
        
    # filter out predominately 'worker' pois (30%+ visits are > 6 hour duration)
    visitor_patterns = filter_workers(clean_patterns)
    
    # read and clean panel data for normalization
    panel_files = get_panel_files(msa_str)
    clean_panels = read_home_panel(panel_files)
    
    print("finished file accumulation, starting flows")
    
    # normalize data using state multiplier, aggregate to cbg/date/naics level
    output_data = normalize_and_aggregate(clean_panels,census_data,visitor_patterns)
    
    print("finished with flows, now writing %s" %msa_str)
    
    # write normalized data to a single csv file
    output_data.coalesce(1)\
    .write\
    .mode("overwrite")\
    .option("mapreduce.fileoutputcommitter.marksuccessfuljobs","false")\
    .option("header","True")\
    .csv("/Users/esrieves/Documents/school/Research/foot_traffic/data/Outputs/normalized_output_data/%s_MSA_18to21_visitor_flows" %msa_str)
    print("finished writing file %s" %msa_str)
    
    end_time = time.time()
    print(end_time - start_time)

NYC
finished file accumulation, starting flows
finished with flows, now writing NYC
finished writing file NYC
164.46701407432556
LA
finished file accumulation, starting flows
finished with flows, now writing LA
finished writing file LA
131.69102907180786
Chicago
finished file accumulation, starting flows
finished with flows, now writing Chicago
finished writing file Chicago
97.09180283546448
Dallas
finished file accumulation, starting flows
finished with flows, now writing Dallas
finished writing file Dallas
103.14730596542358
Houston
finished file accumulation, starting flows
finished with flows, now writing Houston
finished writing file Houston
82.41764283180237
DC
finished file accumulation, starting flows
finished with flows, now writing DC
finished writing file DC
65.92621207237244
Philadelphia
finished file accumulation, starting flows
finished with flows, now writing Philadelphia
finished writing file Philadelphia
56.66780614852905
Miami
finished file accumulation, starting flow